# Рекомендательная система с использованием Deep Learning: Улучшение качества рекомендаций.

**Что требуется:**

- Внедрить методы глубинного обучения для повышения качества рекомендаций (например, использовать нейронные сети).
- Добавить новые признаки, полученные с помощью нейросетей: векторизация текстов постов (Word2Vec, FastText, BERT, и др.), эмбеддинги пользователей или контента.
- Реализовать модель рекомендаций на основе нейронных сетей (например, MLP, CNN, Transformer).
- Поддержать работу с большими таблицами из базы данных (batch загрузка).
- Достичь качества HitRate@5 не менее 0.57.

### Подключение к базе и таблицы с юзерами и постами

In [ ]:
# Импортируем функцию подключения к базе данных из общего модуля common/db_connect.py

import sys
sys.path.append('../common')
from db_connect import get_engine

# Инициализируем объект подключения к базе данных
engine = get_engine()
connection = engine.connect().execution_options(stream_results=True)

In [ ]:
# Посты и топики

import pandas as pd


posts_info = pd.read_sql(
    """SELECT * FROM public.post_text_df""",
    con=connection
)

posts_info

,post_id,text,topic
0,1,UK economy facing major risks\n\nThe UK manufa...,business
1,2,Aids and climate top Davos agenda\n\nClimate c...,business
2,3,Asian quake hits European shares\n\nShares in ...,business
3,4,India power shares jump on debut\n\nShares in ...,business
4,5,Lacroix label bought by US firm\n\nLuxury good...,business
...,...,...,...
7018,7315,"OK, I would not normally watch a Farrelly brot...",movie
7019,7316,I give this movie 2 stars purely because of it...,movie
7020,7317,I cant believe this film was allowed to be mad...,movie
7021,7318,The version I saw of this film was the Blockbu...,movie


### Расчет эмбеддингов постов, кластеризация и сохранение новых признаков в базу данных

In [ ]:
# Импортируем необходимые классы из библиотеки transformers для работы с предобученными языковыми моделями.
# Используем модели BERT, RoBERTa и DistilBERT для генерации эмбеддингов текстов постов.
# Такая архитектура позволяет легко расширять функциональность и использовать разные модели для векторизации текста.

from transformers import AutoTokenizer
from transformers import BertModel  # https://huggingface.co/docs/transformers/model_doc/bert#transformers.BertModel
from transformers import RobertaModel  # https://huggingface.co/docs/transformers/model_doc/roberta#transformers.RobertaModel
from transformers import DistilBertModel  # https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertModel

def get_model(model_name):
    """
    Возвращает токенайзер и предобученную языковую модель (BERT, RoBERTa, DistilBERT) для генерации эмбеддингов текста.
    Использование предобученных моделей позволяет эффективно получать смысловые представления текстов постов,
    что важно для повышения качества работы рекомендательной системы.

    Args:
        model_name (str): название модели ('bert', 'roberta', 'distilbert').

    Returns:
        tokenizer (transformers.PreTrainedTokenizer): токенайзер выбранной модели.
        model (transformers.PreTrainedModel): загруженная предобученная модель.
    """
    assert model_name in ['bert', 'roberta', 'distilbert']

    checkpoint_names = {
        'bert': 'bert-base-cased',  # https://huggingface.co/bert-base-cased
        'roberta': 'roberta-base',  # https://huggingface.co/roberta-base
        'distilbert': 'distilbert-base-cased'  # https://huggingface.co/distilbert-base-cased
    }

    model_classes = {
        'bert': BertModel,
        'roberta': RobertaModel,
        'distilbert': DistilBertModel
    }

    return AutoTokenizer.from_pretrained(checkpoint_names[model_name]), model_classes[model_name].from_pretrained(checkpoint_names[model_name])

/Users/anastasiasharina/Documents/ML_Engineer_karpov/Lessons/Module_3_Deep_Learning/final_project/final_project_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Загружаем предобученный токенайзер и языковую модель DistilBERT для генерации эмбеддингов текста постов.
# Использование современных трансформерных моделей позволяет получить более качественные смысловые представления текстов,
# что существенно повышает эффективность рекомендательной системы.

tokenizer, model = get_model('distilbert')

In [ ]:
# Создаем датасет для постов и настраиваем загрузчик данных для обучения/инференса модели.
# Используем кастомный класс PostDataset, который преобразует тексты постов в формат, подходящий для входа в трансформерную модель (batch_encode_plus).
# Такой подход позволяет удобно работать с большими текстовыми коллекциями и обеспечивает совместимость с пайплайном глубинного обучения.
# DataCollatorWithPadding автоматически выравнивает длины входных последовательностей в батче, упрощая обработку данных.
# DataLoader организует эффективную подачу данных в модель, поддерживает батчинг и случайное перемешивание, что важно для качества обучения.

from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

class PostDataset(Dataset):
    def __init__(self, texts, tokenizer):
        super().__init__()

        self.texts = tokenizer.batch_encode_plus(
            texts,
            add_special_tokens=True,
            return_token_type_ids=False,
            return_tensors='pt',
            truncation=True,
            padding=True
        )
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        return {
            'input_ids': self.texts['input_ids'][idx],
            'attention_mask': self.texts['attention_mask'][idx]
        }

    def __len__(self):
        return len(self.texts['input_ids'])
    
# Формируем датасет для постов, используя подготовленные тексты и токенайзер
dataset = PostDataset(posts_info['text'].values.tolist(), tokenizer)

# Создаем объект для автоматического паддинга батчей
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Настраиваем DataLoader для эффективной загрузки данных в модель
loader = DataLoader(dataset, batch_size=32, collate_fn=data_collator, pin_memory=True, shuffle=False)

In [ ]:
import torch
from tqdm import tqdm

# Получаем эмбеддинги постов из модели в режиме инференса
@torch.inference_mode()
def get_embeddings_labels(model, loader):
    model.eval()

    total_embeddings = []
    for batch in tqdm(loader):
        batch = {key: batch[key].to(device) for key in ['attention_mask', 'input_ids']}
        embeddings = model(**batch)['last_hidden_state'][:, 0, :]
        total_embeddings.append(embeddings.cpu())

    return torch.cat(total_embeddings, dim=0)

In [ ]:
# Определяем устройство для вычислений (GPU или CPU) и переносим модель на выбранное устройство для MacBook, к примеру

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)
if torch.cuda.is_available():
    print(torch.cuda.get_device_name())
else:
    print("CUDA недоступна, используется CPU")
model = model.to(device)

cpu
CUDA недоступна, используется CPU


In [ ]:
# Получаем и сохраняем эмбеддинги постов в numpy-массив для дальнейшего анализа и обработки
embeddings = get_embeddings_labels(model, loader).numpy()

embeddings

  0%|          | 0/220 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/anastasiasharina/Documents/ML_Engineer_karpov/Lessons/Module_3_Deep_Learning/final_project/final_project_env/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
100%|██████████| 220/220 [16:37<00:00,  4.53s/it]


array([[ 3.63150686e-01,  4.89376299e-02, -2.64080763e-01, ...,
        -1.41593412e-01,  1.59179568e-02,  9.19145532e-05],
       [ 2.36416206e-01, -1.59501165e-01, -3.27798396e-01, ...,
        -2.89936483e-01,  1.19365096e-01, -1.62354251e-03],
       [ 3.75191599e-01, -1.13944054e-01, -2.40547329e-01, ...,
        -3.38919550e-01,  5.86937666e-02, -2.12659929e-02],
       ...,
       [ 3.40382546e-01,  6.64920509e-02, -1.63184628e-01, ...,
        -8.65627378e-02,  2.03403652e-01,  3.20906267e-02],
       [ 4.32092071e-01,  1.10913701e-02, -1.17306247e-01, ...,
         7.54015148e-02,  1.02739796e-01,  1.52742993e-02],
       [ 3.04277718e-01, -7.62156248e-02, -6.77586198e-02, ...,
        -5.43491282e-02,  2.44383678e-01, -1.41485240e-02]], dtype=float32)

In [ ]:
# Кластеризуем тексты

from sklearn.decomposition import PCA

centered = embeddings - embeddings.mean()

pca = PCA(n_components=50)
pca_decomp = pca.fit_transform(centered)

In [ ]:
# Кластеризация постов по их эмбеддингам с помощью алгоритма KMeans.
# Получаем номера кластеров для каждого поста и расстояния до центров кластеров — эти признаки можно использовать в рекомендательной системе.

from sklearn.cluster import KMeans

n_clusters = 15

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(pca_decomp)

posts_info['TextCluster'] = kmeans.labels_

dists_columns = [f'DistanceToCluster_{i}' for i in range(n_clusters)]

dists_df = pd.DataFrame(
    data=kmeans.transform(pca_decomp),
    columns=dists_columns
)

dists_df.head()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


,DistanceToCluster_0,DistanceToCluster_1,DistanceToCluster_2,DistanceToCluster_3,DistanceToCluster_4,DistanceToCluster_5,DistanceToCluster_6,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14
0,3.375040,3.457623,3.605905,3.575243,1.755076,3.418822,2.822371,3.721285,2.198266,2.992362,3.446769,3.385027,3.459906,2.349854,3.409753
1,3.327333,3.130830,3.346010,3.370129,1.715311,3.328216,2.549479,3.522700,2.213239,2.841192,2.986242,3.371500,3.225497,2.318709,3.217581
2,3.263286,3.122370,3.338438,3.504163,1.606976,3.348444,2.873301,3.523672,3.010294,3.051372,2.971890,3.499064,3.379388,2.380095,3.286862
3,3.518352,3.786849,3.790487,4.184075,2.343901,3.732917,3.368316,3.020765,3.372246,3.255833,3.721216,3.743400,4.018403,2.804401,3.698213
4,3.032044,2.763682,3.021242,3.391112,1.686560,2.807227,2.120789,3.277803,2.904272,2.644346,2.636745,2.796635,3.175100,2.004909,2.836604


In [ ]:
# Добавляем признаки кластерных расстояний к информации о постах и очищаем датафрейм от лишних столбцов

posts_info = pd.concat((posts_info, dists_df), axis=1)
posts_info.drop(["text"], axis=1, inplace=True)

posts_info

,post_id,topic,TextCluster,DistanceToCluster_0,DistanceToCluster_1,DistanceToCluster_2,DistanceToCluster_3,DistanceToCluster_4,DistanceToCluster_5,DistanceToCluster_6,DistanceToCluster_7,DistanceToCluster_8,DistanceToCluster_9,DistanceToCluster_10,DistanceToCluster_11,DistanceToCluster_12,DistanceToCluster_13,DistanceToCluster_14
0,1,business,4,3.375040,3.457623,3.605905,3.575243,1.755076,3.418822,2.822371,3.721285,2.198266,2.992362,3.446769,3.385027,3.459906,2.349854,3.409753
1,2,business,4,3.327333,3.130830,3.346010,3.370129,1.715311,3.328216,2.549479,3.522700,2.213239,2.841192,2.986242,3.371500,3.225497,2.318709,3.217581
2,3,business,4,3.263286,3.122370,3.338438,3.504163,1.606976,3.348444,2.873301,3.523672,3.010294,3.051372,2.971890,3.499064,3.379388,2.380095,3.286862
3,4,business,4,3.518352,3.786849,3.790487,4.184075,2.343901,3.732917,3.368316,3.020765,3.372246,3.255833,3.721216,3.743400,4.018403,2.804401,3.698213
4,5,business,4,3.032044,2.763682,3.021242,3.391112,1.686560,2.807227,2.120789,3.277803,2.904272,2.644346,2.636745,2.796635,3.175100,2.004909,2.836604
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7018,7315,movie,5,3.133397,1.819893,2.936863,3.502792,2.946092,1.279949,2.335207,2.336890,3.341273,2.820673,3.053946,1.791386,3.397589,2.742617,1.995289
7019,7316,movie,5,2.931358,1.841188,2.586768,3.533731,2.942570,0.939888,2.232086,2.199647,3.178991,2.509073,3.200092,1.431734,3.338599,2.457433,1.772952
7020,7317,movie,5,2.834631,1.986115,2.369662,3.562661,3.179166,1.496023,2.447835,2.627714,3.395509,2.542421,3.159134,2.013413,3.470815,2.817432,2.163502
7021,7318,movie,11,3.431674,1.529222,3.289869,3.502677,3.193173,1.488463,2.315337,1.988967,3.433081,3.110392,3.218611,1.040889,3.426992,2.997247,1.833040


In [ ]:
# Очищаем память, чтобы все влезло

model.cpu()

del model
del tokenizer

del dataset
del loader

del embeddings
del centered
del pca
del pca_decomp

In [19]:
import gc

gc.collect()

0

In [ ]:
# Сохраняем датафрейм с признаками постов в базу данных

posts_info.to_sql(
    "anastasia_sharina_77",
    con=engine,
    schema="public",
    if_exists='replace'
)

23

### Теперь приступаем к обработке действий, обучению и сохранению модели

In [ ]:
# Загружаем выборку из 9 миллионов взаимодействий "view" между пользователями и постами из базы данных.
# В запросе сразу соединяем таблицы feed_data и user_data для получения пользовательских признаков,
# что ускоряет последующую обработку данных для рекомендательной системы.

feed_data = pd.read_sql(
    """
    SELECT
        post_id,
        cast(extract(hour from timestamp) as int) as hour,
        cast(extract(month from timestamp) as int) as month,
        gender,
        age,
        country,
        city,
        exp_group,
        os,
        source,
        target
    FROM public.feed_data JOIN public.user_data ON public.feed_data.user_id = public.user_data.user_id
    WHERE action = 'view'
    LIMIT 9000000
    """,
    con=connection
)

feed_data.head()

,post_id,hour,month,gender,age,country,city,exp_group,os,source,target
0,4223,16,10,0,39,Russia,Moscow,4,Android,organic,0
1,6738,16,10,0,39,Russia,Moscow,4,Android,organic,0
2,5123,16,10,0,39,Russia,Moscow,4,Android,organic,0
3,2691,16,10,0,39,Russia,Moscow,4,Android,organic,0
4,5372,16,10,0,39,Russia,Moscow,4,Android,organic,0


In [ ]:
# Импортируем CatBoostClassifier для обучения модели и tqdm для отображения прогресса.
from catboost import CatBoostClassifier, Pool
from tqdm import tqdm

# Определяем категориальные признаки для модели CatBoost.
object_cols = [
    'topic', 'TextCluster', 'gender', 'country',
    'city', 'exp_group', 'hour', 'month',
    'os', 'source'
]

# Инициализируем модель CatBoost с базовыми параметрами для обучения на табличных данных.
catboost = CatBoostClassifier(
    iterations=200,
    learning_rate=1,
    depth=2,
    random_seed=12345612,
    thread_count=-1,
    task_type="CPU"
)

# Объединяем фичи постов с основной таблицей взаимодействий по post_id.
feed_data = pd.merge(
    feed_data,
    posts_info,
    on='post_id',
    how='left'
)

# Удаляем идентификатор поста, чтобы не использовать его напрямую как признак.
feed_data.drop(['post_id'], axis=1, inplace=True)

# Формируем матрицу признаков для обучения модели.
X = feed_data.drop(['target'], axis=1)

# Выводим типы и названия признаков для контроля корректности данных перед обучением.
print(X.dtypes)
print(X.columns)

hour                      int64
month                     int64
gender                    int64
age                       int64
country                  object
city                     object
exp_group                 int64
os                       object
source                   object
topic                    object
TextCluster               int32
DistanceToCluster_0     float32
DistanceToCluster_1     float32
DistanceToCluster_2     float32
DistanceToCluster_3     float32
DistanceToCluster_4     float32
DistanceToCluster_5     float32
DistanceToCluster_6     float32
DistanceToCluster_7     float32
DistanceToCluster_8     float32
DistanceToCluster_9     float32
DistanceToCluster_10    float32
DistanceToCluster_11    float32
DistanceToCluster_12    float32
DistanceToCluster_13    float32
DistanceToCluster_14    float32
dtype: object
Index(['hour', 'month', 'gender', 'age', 'country', 'city', 'exp_group', 'os',
       'source', 'topic', 'TextCluster', 'DistanceToCluster_0',
       'Dis

In [26]:
# Обучаем модель
catboost.fit(X=X, y=feed_data['target'], cat_features=object_cols)

# Сохраняем модель
catboost.save_model(
    'model.cbm',
    format="cbm"
)

0:	learn: 0.3644995	total: 5.67s	remaining: 18m 47s
1:	learn: 0.3578155	total: 12s	remaining: 19m 48s
2:	learn: 0.3561826	total: 20.2s	remaining: 22m 9s
3:	learn: 0.3554923	total: 32s	remaining: 26m 8s
4:	learn: 0.3545095	total: 41.4s	remaining: 26m 54s
5:	learn: 0.3543314	total: 1m 1s	remaining: 33m 10s
6:	learn: 0.3542250	total: 1m 13s	remaining: 33m 59s
7:	learn: 0.3531892	total: 1m 21s	remaining: 32m 42s
8:	learn: 0.3529023	total: 1m 29s	remaining: 31m 48s
9:	learn: 0.3520800	total: 1m 38s	remaining: 31m 4s
10:	learn: 0.3518138	total: 1m 48s	remaining: 31m 4s
11:	learn: 0.3512752	total: 2m 1s	remaining: 31m 46s
12:	learn: 0.3510278	total: 2m 29s	remaining: 35m 56s
13:	learn: 0.3509514	total: 3m	remaining: 39m 58s
14:	learn: 0.3508172	total: 3m 22s	remaining: 41m 38s
15:	learn: 0.3504355	total: 3m 37s	remaining: 41m 37s
16:	learn: 0.3503290	total: 3m 52s	remaining: 41m 47s
17:	learn: 0.3502373	total: 4m 14s	remaining: 42m 57s
18:	learn: 0.3500583	total: 4m 26s	remaining: 42m 22s
19: